### Chatbot

In [1]:
import random 

import joblib

import sys
import os


import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from tensorflow.keras.models import Model

import scipy

In [2]:
df = pd.read_excel(r"database_intents.xlsx")

In [3]:
# Usar get_dummies en vez de las 3 funciones
df_oh = pd.concat([df, pd.get_dummies(df['Intent type'])], axis = 1)

In [4]:
df_train, df_test = train_test_split(df_oh, train_size = 0.7, test_size = 0.3, random_state = 42,
                                    shuffle = True, stratify = df_oh['Intent type'])

In [5]:
stopwords = []

In [6]:
cv = TfidfVectorizer(
    stop_words= stopwords,
    ngram_range=(1, 4),
    strip_accents='ascii',
    max_df=0.99,
    min_df=0,
    max_features=100
    )
cv.fit(df_train["Sentence"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.99, max_features=100, min_df=0,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=[], strip_accents='ascii', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
import pickle

with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(cv, file)

In [8]:
def processing(df, pretreatment = False, Tfidf = True, cv = None, stopwords = []):
  # Normalizamos y limpiamos el corpus 
  if pretreatment == True:
    df["Sentence"] = df['Sentence'].apply(lambda x: word_treatment(x))
    print("El corpus ha sido pretratado")
    
     # Transformamos nuestro corpus a un vector Tfidf
  if Tfidf == True:

    if cv == None:
      cv = TfidfVectorizer(
        stop_words= stopwords,
        ngram_range=(1, 4),
        strip_accents='ascii',
        max_df=0.99,
        min_df=0,
        max_features=100
      )
      cv.fit(df["Sentence"])
      X = cv.transform(df["Sentence"])
      print("Se ha realizado una vectorización Tfidf")
      return df, X, cv

    else:
      X = cv.transform(df["Sentence"])
      #print("Se ha realizado una vectorización Tfidf basado en el corpus suministrado por cv")
    return df, X
  else:
    return df


In [9]:
df_train, X_train, cv = processing(df_train)
df_test, X_test = processing(df_test, cv = cv)

Se ha realizado una vectorización Tfidf


In [10]:
y_train = df_train[["Greeting","Search","Suggestions"]]
y_test = df_test[["Greeting","Search","Suggestions"]]

In [11]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.7, 
test_size=0.3, random_state=42, shuffle=True, stratify=df_train["Intent type"])

In [12]:
shape= X_train.shape[1]

X_train.sort_indices()
X_validation.sort_indices()

In [13]:
def mlp_greeting(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [14]:
mlp_greeting = mlp_greeting(shape)
history = mlp_greeting.fit(X_train, np.asarray(y_train["Greeting"]).reshape(-1,1),
                  validation_data = (X_validation, np.asarray(y_validation["Greeting"]).reshape(-1,1)),
    epochs = 40,
    workers = 2, use_multiprocessing = True, verbose = 2)

Epoch 1/40


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.7457 - accuracy: 0.5085 - val_loss: 0.6714 - val_accuracy: 0.7308
Epoch 2/40
2/2 - 0s - loss: 0.6620 - accuracy: 0.7627 - val_loss: 0.6526 - val_accuracy: 0.7308
Epoch 3/40
2/2 - 0s - loss: 0.6422 - accuracy: 0.7627 - val_loss: 0.6363 - val_accuracy: 0.7308
Epoch 4/40
2/2 - 0s - loss: 0.6234 - accuracy: 0.7627 - val_loss: 0.6224 - val_accuracy: 0.7308
Epoch 5/40
2/2 - 0s - loss: 0.6079 - accuracy: 0.7627 - val_loss: 0.6110 - val_accuracy: 0.7308
Epoch 6/40
2/2 - 0s - loss: 0.5920 - accuracy: 0.7627 - val_loss: 0.6021 - val_accuracy: 0.7308
Epoch 7/40
2/2 - 0s - loss: 0.5835 - accuracy: 0.7627 - val_loss: 0.5950 - val_accuracy: 0.7308
Epoch 8/40
2/2 - 0s - loss: 0.5741 - accuracy: 0.7627 - val_loss: 0.5899 - val_accuracy: 0.7308
Epoch 9/40
2/2 - 0s - loss: 0.5643 - accuracy: 0.7627 - val_loss: 0.5864 - val_accuracy: 0.7308
Epoch 10/40
2/2 - 0s - loss: 0.5601 - accuracy: 0.7627 - val_loss: 0.5842 - val_accuracy: 0.7308
Epoch 11/40
2/2 - 0s - loss: 0.5559 - accuracy: 0.

In [15]:
def mlp_search(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [16]:
mlp_search = mlp_search(shape)
history = mlp_search.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs = 100,
    workers = 2, use_multiprocessing= True, verbose = 2)

Epoch 1/100


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense_5/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense_5/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense_5/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.6763 - accuracy: 0.7458 - val_loss: 0.6333 - val_accuracy: 0.7692
Epoch 2/100
2/2 - 0s - loss: 0.5977 - accuracy: 0.7627 - val_loss: 0.5291 - val_accuracy: 0.7692
Epoch 3/100
2/2 - 0s - loss: 0.5512 - accuracy: 0.7627 - val_loss: 0.4909 - val_accuracy: 0.7692
Epoch 4/100
2/2 - 0s - loss: 0.5027 - accuracy: 0.7627 - val_loss: 0.4865 - val_accuracy: 0.7692
Epoch 5/100
2/2 - 0s - loss: 0.4038 - accuracy: 0.7627 - val_loss: 0.4892 - val_accuracy: 0.7692
Epoch 6/100
2/2 - 0s - loss: 0.4217 - accuracy: 0.7627 - val_loss: 0.4762 - val_accuracy: 0.7692
Epoch 7/100
2/2 - 0s - loss: 0.4076 - accuracy: 0.7627 - val_loss: 0.4624 - val_accuracy: 0.7692
Epoch 8/100
2/2 - 0s - loss: 0.3945 - accuracy: 0.7627 - val_loss: 0.4581 - val_accuracy: 0.7692
Epoch 9/100
2/2 - 0s - loss: 0.3580 - accuracy: 0.7627 - val_loss: 0.4669 - val_accuracy: 0.7692
Epoch 10/100
2/2 - 0s - loss: 0.3854 - accuracy: 0.7627 - val_loss: 0.4940 - val_accuracy: 0.7692
Epoch 11/100
2/2 - 0s - loss: 0.3302 - ac

Epoch 85/100
2/2 - 0s - loss: 0.0713 - accuracy: 0.9831 - val_loss: 1.2120 - val_accuracy: 0.9231
Epoch 86/100
2/2 - 0s - loss: 0.2059 - accuracy: 0.9153 - val_loss: 1.2081 - val_accuracy: 0.9231
Epoch 87/100
2/2 - 0s - loss: 0.1170 - accuracy: 0.9661 - val_loss: 1.2299 - val_accuracy: 0.9231
Epoch 88/100
2/2 - 0s - loss: 0.0690 - accuracy: 0.9831 - val_loss: 1.2521 - val_accuracy: 0.9231
Epoch 89/100
2/2 - 0s - loss: 0.2422 - accuracy: 0.9153 - val_loss: 1.2630 - val_accuracy: 0.9231
Epoch 90/100
2/2 - 0s - loss: 0.0735 - accuracy: 0.9831 - val_loss: 1.2829 - val_accuracy: 0.9231
Epoch 91/100
2/2 - 0s - loss: 0.1432 - accuracy: 0.9492 - val_loss: 1.2962 - val_accuracy: 0.9231
Epoch 92/100
2/2 - 0s - loss: 0.1203 - accuracy: 0.9661 - val_loss: 1.3420 - val_accuracy: 0.9231
Epoch 93/100
2/2 - 0s - loss: 0.1383 - accuracy: 0.9492 - val_loss: 1.3758 - val_accuracy: 0.9231
Epoch 94/100
2/2 - 0s - loss: 0.1513 - accuracy: 0.9492 - val_loss: 1.3972 - val_accuracy: 0.9231
Epoch 95/100
2/2 - 0

In [17]:
def mlp_suggestion(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.01)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer = opt)
# return our model
    return model

In [18]:
mlp_suggestion = mlp_suggestion(shape)
history = mlp_suggestion.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs = 250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Epoch 1/250


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_2/dense_10/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_2/dense_10/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_2/dense_10/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.7740 - accuracy: 0.3898 - val_loss: 0.7141 - val_accuracy: 0.4231
Epoch 2/250
2/2 - 0s - loss: 0.7037 - accuracy: 0.5932 - val_loss: 0.6866 - val_accuracy: 0.7308
Epoch 3/250
2/2 - 0s - loss: 0.6884 - accuracy: 0.7119 - val_loss: 0.6751 - val_accuracy: 0.7308
Epoch 4/250
2/2 - 0s - loss: 0.6720 - accuracy: 0.7966 - val_loss: 0.6682 - val_accuracy: 0.8462
Epoch 5/250
2/2 - 0s - loss: 0.6529 - accuracy: 0.8475 - val_loss: 0.6627 - val_accuracy: 0.8462
Epoch 6/250
2/2 - 0s - loss: 0.6462 - accuracy: 0.8475 - val_loss: 0.6575 - val_accuracy: 0.8462
Epoch 7/250
2/2 - 0s - loss: 0.6501 - accuracy: 0.8136 - val_loss: 0.6516 - val_accuracy: 0.8077
Epoch 8/250
2/2 - 0s - loss: 0.6414 - accuracy: 0.8475 - val_loss: 0.6464 - val_accuracy: 0.7692
Epoch 9/250
2/2 - 0s - loss: 0.6048 - accuracy: 0.8983 - val_loss: 0.6421 - val_accuracy: 0.8077
Epoch 10/250
2/2 - 0s - loss: 0.6133 - accuracy: 0.8814 - val_loss: 0.6394 - val_accuracy: 0.8077
Epoch 11/250
2/2 - 0s - loss: 0.5969 - ac

Epoch 85/250
2/2 - 0s - loss: 0.1665 - accuracy: 1.0000 - val_loss: 1.9311 - val_accuracy: 0.8462
Epoch 86/250
2/2 - 0s - loss: 0.1843 - accuracy: 0.9831 - val_loss: 1.9147 - val_accuracy: 0.8462
Epoch 87/250
2/2 - 0s - loss: 0.1538 - accuracy: 1.0000 - val_loss: 1.9018 - val_accuracy: 0.8462
Epoch 88/250
2/2 - 0s - loss: 0.1581 - accuracy: 1.0000 - val_loss: 1.9097 - val_accuracy: 0.8462
Epoch 89/250
2/2 - 0s - loss: 0.1504 - accuracy: 1.0000 - val_loss: 1.9184 - val_accuracy: 0.8462
Epoch 90/250
2/2 - 0s - loss: 0.1484 - accuracy: 1.0000 - val_loss: 1.9262 - val_accuracy: 0.8462
Epoch 91/250
2/2 - 0s - loss: 0.1539 - accuracy: 1.0000 - val_loss: 1.9533 - val_accuracy: 0.8462
Epoch 92/250
2/2 - 0s - loss: 0.1985 - accuracy: 0.9661 - val_loss: 2.0069 - val_accuracy: 0.8462
Epoch 93/250
2/2 - 0s - loss: 0.1441 - accuracy: 1.0000 - val_loss: 2.0618 - val_accuracy: 0.8462
Epoch 94/250
2/2 - 0s - loss: 0.1457 - accuracy: 1.0000 - val_loss: 2.1213 - val_accuracy: 0.8462
Epoch 95/250
2/2 - 0

Epoch 168/250
2/2 - 0s - loss: 0.0708 - accuracy: 1.0000 - val_loss: 3.5084 - val_accuracy: 0.8462
Epoch 169/250
2/2 - 0s - loss: 0.0700 - accuracy: 1.0000 - val_loss: 3.5510 - val_accuracy: 0.8462
Epoch 170/250
2/2 - 0s - loss: 0.1108 - accuracy: 0.9831 - val_loss: 3.5859 - val_accuracy: 0.8462
Epoch 171/250
2/2 - 0s - loss: 0.0703 - accuracy: 1.0000 - val_loss: 3.5987 - val_accuracy: 0.8462
Epoch 172/250
2/2 - 0s - loss: 0.1110 - accuracy: 0.9831 - val_loss: 3.5944 - val_accuracy: 0.8462
Epoch 173/250
2/2 - 0s - loss: 0.0679 - accuracy: 1.0000 - val_loss: 3.5875 - val_accuracy: 0.8462
Epoch 174/250
2/2 - 0s - loss: 0.0674 - accuracy: 1.0000 - val_loss: 3.5816 - val_accuracy: 0.8462
Epoch 175/250
2/2 - 0s - loss: 0.0669 - accuracy: 1.0000 - val_loss: 3.5767 - val_accuracy: 0.8462
Epoch 176/250
2/2 - 0s - loss: 0.1023 - accuracy: 0.9831 - val_loss: 3.4153 - val_accuracy: 0.8462
Epoch 177/250
2/2 - 0s - loss: 0.0659 - accuracy: 1.0000 - val_loss: 3.2849 - val_accuracy: 0.8462
Epoch 178/

In [19]:
def mlp_farewell(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [20]:
mlp_farewell = mlp_farewell(shape)
history = mlp_farewell.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs = 50,
    workers = 2, use_multiprocessing= True, verbose = 2)

Epoch 1/50


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_3/dense_15/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_3/dense_15/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_3/dense_15/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.7075 - accuracy: 0.5254 - val_loss: 0.6675 - val_accuracy: 0.7692
Epoch 2/50
2/2 - 0s - loss: 0.6642 - accuracy: 0.7627 - val_loss: 0.6449 - val_accuracy: 0.7692
Epoch 3/50
2/2 - 0s - loss: 0.6385 - accuracy: 0.7627 - val_loss: 0.6253 - val_accuracy: 0.7692
Epoch 4/50
2/2 - 0s - loss: 0.6072 - accuracy: 0.8305 - val_loss: 0.6079 - val_accuracy: 0.7692
Epoch 5/50
2/2 - 0s - loss: 0.5818 - accuracy: 0.8136 - val_loss: 0.5924 - val_accuracy: 0.7692
Epoch 6/50
2/2 - 0s - loss: 0.5611 - accuracy: 0.8136 - val_loss: 0.5761 - val_accuracy: 0.7692
Epoch 7/50
2/2 - 0s - loss: 0.5241 - accuracy: 0.8305 - val_loss: 0.5569 - val_accuracy: 0.7692
Epoch 8/50
2/2 - 0s - loss: 0.5293 - accuracy: 0.8136 - val_loss: 0.5463 - val_accuracy: 0.7692
Epoch 9/50
2/2 - 0s - loss: 0.4620 - accuracy: 0.8644 - val_loss: 0.5388 - val_accuracy: 0.7692
Epoch 10/50
2/2 - 0s - loss: 0.4581 - accuracy: 0.8475 - val_loss: 0.5316 - val_accuracy: 0.7692
Epoch 11/50
2/2 - 0s - loss: 0.4291 - accuracy: 0.

In [21]:
# mlp_greeting.save('model_greetings.h5')
# mlp_search.save('model_search.h5')
# mlp_suggestion.save('model_suggestion.h5')
# mlp_farewell.save('model_farewell.h5')

# CHATBOT TEST

In [95]:
mlp_greeting = keras.models.load_model('model_greetings.h5')
mlp_search = keras.models.load_model('model_search.h5')
mlp_suggestion = keras.models.load_model('model_suggestion.h5')
mlp_farewell = keras.models.load_model('model_farewell.h5')
# cv = joblib.load("vectorizer.pkl")

In [83]:
def chatbot():
    
    input_text = input()
    
    test = pd.DataFrame(data = {'Sentence': [input_text]})
    df_test_proc, test_proc = processing(test, cv = cv)
    scipy.sparse.csr_matrix.sort_indices(test_proc)
    
    gret_prob = mlp_greeting.predict(test_proc)
    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    farewell_prob = mlp_farewell.predict(test_proc)
    
    probs = [gret_prob, search_prob, sugg_prob, farewell_prob]
    print(probs)
    idx = np.argmax(probs)
    
    if idx == 0:
        print("Esto es un saludo")
    elif idx == 1:
        print("Esto es una búsqueda")
    elif idx == 2:
        print("Esto es una sugerencia")
    else:
        print("Esto es una despedida")
        
#     print('¿Hemos acertado?')
    
#     respuesta = input()
#     if (respuesta == 'No' or respuesta == 'no'):
#         probs = np.delete(probs, idx)
#         idx_2 = np.argmax(probs)
        
#         if idx == 0:
#             if idx_2 == 0:
#                 print("Esto es una búsqueda")
#             else:
#                 print("Esto es una sugerencia")
#         elif idx == 1:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una sugerencia")
#         else:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una búsqueda")
#     else:
#         print('¡Genial! ¡Hemos acertado!')
    

In [96]:
chatbot()

hi
[array([[0.8040894]], dtype=float32), array([[0.00038779]], dtype=float32), array([[3.612887e-05]], dtype=float32), array([[0.00043699]], dtype=float32)]
Esto es un saludo


In [81]:
chatbot()

hi
[array([[0.7962549]], dtype=float32), array([[0.01487446]], dtype=float32), array([[0.0009267]], dtype=float32), array([[4.4460567e-06]], dtype=float32)]
Esto es un saludo


In [ ]:
chatbot()

In [ ]:
chatbot()

* Correctamente
 * La mayoría de saludos
 * I want to know about Data Science (búsqueda)
 * I want to know things connected to la Sagrada Familia (sugerencia)
 * Watcha doing'? (saludo)
 * Tell me things related to Data Science (sugerencia)

* Incorrectamente
 * What is Data Science (sugerencia)
 * Tell me things related to Napoleon (saludo)
 * Who is Cristobal Colon (saludo)



In [ ]:
chatbot()

* Opciones
 * Mostrar un comando *!options* que te de las opciones para hacer.
 * Detectar un intent
 * Mostrarlo al inicio


* Headers
 * Dar las opciones una vez se detecta intent de búsqueda búsqueda
   * *What is data science?* Y mostrarle los heades para qué elija qué queire saber 
 * Intent nuevo de header

In [ ]:
chatbot()

In [ ]:
def Intent_detection_function(keyboard):
    gret_prob = mlp_greeting.predict_proba(test_proc)
    search_prob = mlp_search.predict_proba(test_proc)
    sugg_prob = mlp_suggestion.predict_proba(test_proc)
    farewell_prob = mlp_farewell.predict_proba(test_proc)
    cv = joblib.load("vectorizer.pkl")


    input_text = pd.DataFrame(data={'Sentence': [keyboard]})
    _, test_proc = cv.transform(input_text['Sentence'])
    #df_test_proc, test_proc = processing(test, cv=cv)

    test_proc = test_proc.toarray()

    gret_prob = mlp_greeting.predict(test_proc)

    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    fare_prob = mlp_farewell.predict(test_proc)


    probs = [gret_prob, search_prob, sugg_prob, fare_prob]
    print(probs)
    idx = np.argmax(probs)

    if idx == 0:
        intent = "Greeting"
        keyword = None
    elif idx == 1:
        intent = "Search"
        keyword = getEntities(keyboard)
    elif idx == 2:
        intent = "Suggestions"
        keyword = getEntities(keyboard)
    elif idx == 3:
        intent = "Farewell"
        keyword = None

    return intent, keyword

In [ ]:
Intent_detection_function()